In [65]:
import requests as rq
from bs4 import BeautifulSoup as bs
from bs4.element import Comment
import codecs
import collections
import pandas as pd
import numpy as np
import scipy as scp
import re

In [ ]:
def getPage(url, name):
    page = rq.get(url)
    print(page)
    f = codecs.open(name, 'w', 'latin1')
    f.write(page.text)
    f.close
    # parsed_page = BeautifulSoup(page.text,"html.parser")

# url = "https://www.lojasrenner.com.br/p/jaqueta-bomber-ultra-leve/-/A-544333014-br.lr?sku=544590918"
# getPage(url, "page1.html")

In [2]:
def beautiful(arquivo): 
    parsed_page = bs(open(arquivo),"html.parser")
#     print(dir(parsed_page))
#     print(parsed_page.prettify()[:1000])
    print (parsed_page.div)
    return parsed_page

page = beautiful("page1.html")

<div class="oo-loader-default"></div>


In [3]:
page.title

<title>Jaqueta Bomber Ultra Leve - Renner</title>

In [4]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
#     soup = bs(body, 'html.parser')
    texts = body.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

pageText = text_from_html(page)

In [5]:
pageList = pageText.split(" ")
pageList = filter(lambda x: len(x)>0, pageList)
# values = set(x for x in pageList)
# print values
c = collections.Counter(pageList)
print c.keys()

[u'confira', u'BUSQUE', u'Frequentes', u'Marcas', u'CPF', u'Novidades', u'bomber', u'Praia', u'Klein', u'Acompanhe', u'pedido,', u'1.76', u'Blusas', u'Aceite', u'Blazers', u'Brindes', u'Western', u'Clique', u'Villanova,', u'Marca:', u'destes', u'N\xe3o', u'Calvin', u'GG', u'P', u'Plus', u'seja,', u'fale', u'Just', u'product.titleProduct', u'Cadastrar', u'meses', u'10X*', u'seguran\xe7a', u'reclama\xe7\xf5es,', u'Genuine', u'10x', u'Ofertas', u'situa\xe7\xe3o', u'**Para', u'Camisas', u'habilitado', u'G', u'sua', u'+', u'Blus\xf5es', u'Veludo', u'Militar', u'*sobrenome', u'Cart\xe3o.', u'Unissex', u'n\xfamero)', u'SC', u'Gravatas', u'Marte', u'\xd3culos', u'ME', u'instru\xe7\xf5es', u'Presentinhos', u'Listras', u'sujeita', u'digite', u'SOCIAL', u'Jardim', u'Cart\xe3o', u'Style', u'Sand\xe1lias', u'Pulseiras', u'Duomo', u'Frete', u'suggest.label', u'HOME', u'total', u'indispon\xedvel.', u'reservados', u'&', u'Sapatilhas', u'Composi\xe7\xe3o:', u'Casio', u'todas', u'6', u'SUGEST\xd5ES', u'

In [60]:
data_set = pd.read_csv("sites.csv")

In [61]:
data_set.describe()

,url,anuncio
count,224,224
unique,224,2
top,https://www.gregory.com.br/camisa-alongada-ver...,não
freq,1,115


In [62]:
# data_set = data_set.drop_duplicates(subset= ['url'],keep='first')
# data_set

In [44]:
# sites = data_set.iloc[:,:]
print data_set.url[11]
print data_set.anuncio[11]

https://www.riachuelo.com.br/sueter-malha-basica-12043524
sim


In [64]:
for i, site in enumerate(data_set.url):
    
    print i, site, data_set.anuncio[i]
# data_set.iloc[:,0][23]

0 https://www.riachuelo.com.br/ não
1 https://www.riachuelo.com.br/masculino não
2 https://www.riachuelo.com.br/campanhas/masculino/mar-2018/bermudas-surf não
3 https://www.riachuelo.com.br/bermuda-listras-11828862 não
4 https://www.riachuelo.com.br/bermuda-listra-11835044?src_click=item_page.recs_product_1&id=11835044 não
5 https://www.riachuelo.com.br/kit-meia-3-pares-cano-invisivel-mash-11604506?src_click=item_page.recs_product_2&id=11604506 não
6 https://www.riachuelo.com.br/beleza-e-perfume não
7 https://www.riachuelo.com.br/campanhas/beleza/mar-2018/hugo-boss não
8 https://www.riachuelo.com.br/perfume-masculino-hugo-boss-bottled-tonic-eau-de-toilette-12079332 não
9 https://www.riachuelo.com.br/teens não
10 https://www.riachuelo.com.br/campanhas/teens/mar-2018/combo-camisetas não
11 https://www.riachuelo.com.br/sueter-malha-basica-12043524 sim
12 https://www.riachuelo.com.br/blusa-tricot-listras-12043583 sim
13 https://www.riachuelo.com.br/blusa-basica-manga-longa-12131067 sim
14 

In [75]:
padrao = "((www\.)?[a-zA-Z0-9]+\.[a-zA-Z0-9]+(\.[a-zA-Z0-9]+)*)"
teste = "https://www.riachuelo.com.br/blusa-basica-manga-longa-12131067"
print re.search(padrao, teste)